### ***5***

# LOGISTIC REGRESSION AND BGD + 5-FOLD

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%reset-f

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [ ]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [ ]:
# hypothesis for logistic regression which is the prediction 
def hypothesis(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost(h,insts,m):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c
# update of weight values for logistic regression
def update(w,alpha,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w
def update_stotastic(w,alpha,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i] - alpha*d[i]

  return w

def performance(mat,p):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  p[0]=accuracy
  p[1]=sensitivity
  p[2]=specificity
  return p

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(alpha))
  for i in range(len(alpha)):
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      h = hypothesis(w,insts_tr,m_tr,n)
      w = update(w,alpha[i],insts_tr,h,m_tr,n)
      j_list[j] = cost(h,insts_tr,m_tr)
    c[i] = j_list[itr-1]
    if c[i]< b_min:
      b_min = c[i]
      j_opt = j_list
      w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9700978108989288
sensitivity : 0.9506720430107528
specificity : 0.9686762360446572


# LOGISTIC REGRESSION AND MBG + 5-FOLD

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(alpha))
  m_batch = 20
  for i in range(len(alpha)):
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      np.random.shuffle(insts_tr)
      h = hypothesis(w,insts_tr,m_tr,n)
      w = update(w,alpha[i],insts_tr,h,m_batch,n)
      j_list[j] = cost(h,insts_tr,m_tr)
    c[i] = j_list[itr-1]
    if c[i]< b_min:
      b_min = c[i]
      j_opt = j_list
      w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9208663251047973
sensitivity : 0.8696911205489
specificity : 0.9401397325008402


# LOGISTIC REGRESSION AND SGD + 5-FOLD

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_s[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_s,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.001,0.01,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(alpha))
  for i in range(len(alpha)):
    w = np.random.rand(n)
    j_list = np.ones(itr)
    for j in range(itr):
      index = np.random.randint(0,m_tr) 
      h = hypothesis(w,insts_tr,m_tr,n)
      w = update_stotastic(w,alpha[i],insts_tr,h,index,n)
      j_list[j] = cost(h,insts_tr,m_tr)
    c[i] = j_list[itr-1]
    if c[i]< b_min:
      b_min = c[i]
      j_opt = j_list
      w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9103089582362986
sensitivity : 0.854946611807587
specificity : 0.9271025095198109


# LOGISTIC REGRESSION AND BGD + L2-NORM + 5-FOLD

In [ ]:
%reset-f

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy

In [ ]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [ ]:
# hypothesis for logistic regression which is the prediction 
def hypothesis_l2(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost_l2(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  return c + 0.5*lamb*(np.dot(w,w))
# update of weight values for logistic regression
def update_l2(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] =  w[i]*(1-alpha*lamb) - alpha*d[i]

  return w
def update_stotastic_l2(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = w[i]*(1-alpha*lamb) - alpha*d[i]

  return w

def performance_l2(mat,p):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  p[0]=accuracy
  p[1]=sensitivity
  p[2]=specificity
  return p

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        h = hypothesis_l2(w,insts_tr,m_tr,n)
        w = update_l2(w,a,lamb[i],insts_tr,h,m_tr,n)
        j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l2(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9701443875174661
sensitivity : 0.9402132499164072
specificity : 0.9772008044881973


# LOGISTIC REGRESSION AND MBG + L2-NORM + 5-FOLD

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  m_batch = 20 #size of batch
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        np.random.shuffle(insts_tr)
        h = hypothesis_l2(w,insts_tr,m_tr,n)
        w = update_l2(w,a,lamb[i],insts_tr,h,m_batch,n)
        j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l2(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9050458003415619
sensitivity : 0.8455981854167337
specificity : 0.9258195853660662


# LOGISTIC REGRESSION AND SGD + L2-NORM + 5-FOLD

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_s[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_s,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        index = np.random.randint(m_tr) 
        h = hypothesis_l2(w,insts_tr,m_tr,n)
        w = update_stotastic_l2(w,a,lamb[i],insts_tr,h,index,n)
        j_list[j] = cost_l2(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l2(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l2(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.8962893960565129
sensitivity : 0.8165233317407232
specificity : 0.9322613229064842


# LOGISTIC REGRESSION AND BGD + L1-NORM + 5-FOLD

In [2]:
%reset-f

In [3]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import math
import copy
import warnings

In [6]:
path = '/content/drive/MyDrive/ML/data_q4_q5.xlsx' 
df = pd.read_excel(path)
insts = df.to_numpy()
m= len(insts[:,0]) #No of instinces
for i in range(m):
  insts[i,-1] = (ord(insts[i,-1])-66)/11
ones = np.ones((m,1))
insts = np.append(ones,insts,axis=1)
n = len(insts[0,:])-1 #no of features including f0 n=31
for i in range(1,n,1):
  insts[:,i] = (insts[:,i] - insts[:,i].mean())/insts[:,i].std()


insts_mb = copy.deepcopy(insts)
insts_s = copy.deepcopy(insts)

In [7]:
# hypothesis for logistic regression which is the prediction 
def hypothesis_l1(w,insts,m,n):
  z = np.dot(insts[:,0:n],w.T)
  h = 1/(1+np.exp(-z.astype(np.float32))) #working
  return h

# cost function for logistic regression
def cost_l1(h,insts,lamb,m,w):
  c = 0
  for i in range(m):
    c_te = (-insts_tr[i,-1]*np.log(h[i]) - (1- insts_tr[i,-1])*np.log(1 - h[i]))/m
    if not(np.isnan(c_te)) and not(math.isinf(c_te)):
      c = c+ c_te
  w_abs = np.absolute(w)
  w_abs = np.sum(w_abs)
  return c + 0.5*lamb*(w_abs)
# update of weight values for logistic regression
def update_l1(w,alpha,lamb,insts,h,m,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = np.dot( (h[0:m]- insts[0:m,-1]),insts[0:m,i] )
  for i in range(n):
    w[i] =  (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w
def update_stotastic_l1(w,alpha,lamb,insts,h,index,n):
  d = np.zeros(n)
  for i in range(n):
    d[i] = (h[index]- insts[index,-1])*insts[index,i] 
  for i in range(n):
    w[i] = (w[i]-alpha*lamb*np.sign(w[i])) - alpha*d[i]

  return w

def performance_l1(mat,p):
  accuracy = (mat[0,0]+mat[1,1])/(mat[0,0]+mat[1,1]+mat[0,1]+mat[1,0])
  sensitivity = (mat[1,1])/(mat[1,1]+mat[1,0])
  specificity = (mat[0,0])/(mat[0,1]+mat[0,0])
  p[0]=accuracy
  p[1]=sensitivity
  p[2]=specificity
  return p

In [ ]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        h = hypothesis_l1(w,insts_tr,m_tr,n)
        w = update_l1(w,a,lamb[i],insts_tr,h,m_tr,n)
        j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l1(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9701133364384411
sensitivity : 0.9454939191296212
specificity : 0.9727018094687268


# LOGISTIC REGRESSION AND MBG + L1-NORM + 5-FOLD

In [ ]:
warnings.filterwarnings('ignore')
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_mb[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_mb,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  m_batch = 20 #size of batch
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        np.random.shuffle(insts_tr)
        h = hypothesis_l1(w,insts_tr,m_tr,n)
        w = update_l1(w,a,lamb[i],insts_tr,h,m_batch,n)
        j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l1(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

In [ ]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9156342182890855
sensitivity : 0.847155548898734
specificity : 0.9459272519080371


# LOGISTIC REGRESSION AND SGD + L1-NORM + 5-FOLD

In [8]:
K = 5 # no of folds
avg_accuracy = 0
avg_sensitivity = 0
avg_specificity = 0
for k in range(K):
  insts_te = insts_s[int((0.8-0.2*k)*m):int(m*(1-0.2*k)),:]
  insts_tr = np.delete(insts_s,np.s_[int((0.8-0.2*k)*m):int((1-0.2*k)*m)], axis=0)
  itr = 100
  alpha = np.linspace(0.0001,0.001,10)
  lamb = np.linspace(0,100,10)
  m_tr = len(insts_tr[:,0])
  m_te = len(insts_te[:,0])
  w_opt = np.zeros(n)  #stores the optimal weight value
  j_opt = np.ones(itr) #stores the optimal j value list
  b_min = 10000
  c = np.ones(len(lamb))
  for i in range(len(lamb)):
    for a in alpha:
      w = np.random.rand(n)
      j_list = np.ones(itr)
      for j in range(itr):
        index = np.random.randint(m_tr) 
        h = hypothesis_l1(w,insts_tr,m_tr,n)
        w = update_stotastic_l1(w,a,lamb[i],insts_tr,h,index,n)
        j_list[j] = cost_l1(h,insts_tr,lamb[i],m_tr,w)
      c[i] = j_list[itr-1]
      if c[i]< b_min:
        b_min = c[i]
        j_opt = j_list
        w_opt = w
  m_te = len(insts_te[:,0])
  h_te = hypothesis_l1(w_opt,insts_te,m_te,n)
  y_pre = np.round(h_te)
  y_acu = insts_te[:,-1]
  TruNeg = 0
  Trupos = 0
  FN = 0
  FP = 0
  for i in range(m_te):
    if y_pre[i] == 0:
      if y_acu[i] == 0:
        TruNeg = TruNeg +1
      else:
        FP = FP +1
    else:
      if y_acu[i] == 1:
        Trupos = Trupos +1
      else:
        FN = FN +1
  con_matrix = np.array([[TruNeg,FP],[FN,Trupos]])
  p= np.ones(3)
  p= performance_l1(con_matrix,p)
  avg_accuracy += p[0]
  avg_sensitivity +=p[1]
  avg_specificity += p[2]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()


In [9]:
print('accuracy :',end=' ')
print(avg_accuracy/K)
print('sensitivity :',end=' ')
print(avg_sensitivity/K)
print('specificity :',end=' ')
print(avg_specificity/K)

accuracy : 0.9050458003415619
sensitivity : 0.8376651855271511
specificity : 0.933447167436975
